In [30]:
#POST FUNCTION TEST

import requests
import pandas as pd

#----THIS CODE IS DIRECTLY FROM https://apps.fs.usda.gov/fiadb-api DOCUMENTATION----
#This code writes the function for the API request
# this function will accept a FIADB-API fullreport URL and return dataframes for the estimates as well as subtotals, and totals where available.

def fiadb_api_POST(parameterDictionary):
    # make request
    resp = requests.post(r"https://apps.fs.usda.gov/fiadb-api/fullreport",data=parameterDictionary)
    # parse response to json
    data = resp.json()

    # create output dictionary and populate it with estimate data frames
    outDict = {}
    # append estimates
    outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
    if 'subtotals' in data.keys():
        subT = {}
        for i in data['subtotals'].keys():
            subT[i] = pd.DataFrame(data['subtotals'][i])
        outDict['subtotals'] = subT
        outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
    outDict['metadata'] = data['metadata']
    return outDict

In [33]:
# This code block will format the API request URL based on the chosen state - see markdown above

#Colorado - 2019
statecode = '082019'


In [34]:
# This code block will make the API requests for each estimate type

# 79_SAMPLEDLANDWATER_ACRES
slw_parameters = {'snum':'79', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
sampledlandwater_acres_GET = fiadb_api_POST(slw_parameters)

# 2_FORESTLAND_ACRES
fl_parameters = {'snum':'2', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
forestland_acres_GET = fiadb_api_POST(fl_parameters)

In [35]:
# This code block will format the returned data for each estimate and join them BY COUNTY, and export the final .csv
sampledlandwater_acres = sampledlandwater_acres_GET['estimates']
slw_acres = sampledlandwater_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
slw_79_acres = slw_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '79_SAMPLEDLANDWATER_ACRES'})

forestland_acres = forestland_acres_GET['estimates']
fl_acres = forestland_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
fl_2_acres = fl_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '2_FORESTLAND_ACRES'})

# Inner Join attempt on these two dataframes
CO_data = slw_79_acres.merge(fl_2_acres, how='left', on='STATE_COUNTY_CODE')
CO_data_final = CO_data.fillna(0)